# Imports

In [3]:
#!pip install -U scikit-learn

In [12]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from torch import nn
from numpy import mean
#!pip install igraph
from igraph import Graph as igraphGraph

# Input

In [5]:
transformation = Transformation()

user_number_triangles = 500
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [6]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [7]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [41]:
class DevConv(nn.Module):
    def __init__(self, graph, output_dimension):
        super().__init__()
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = torch.tensor(np.random.random((output_dimension))) #change #passer en tensor
        self.W_theta = torch.tensor(np.array([0.1, 0.1, 0.1]).reshape(1, -1))  # change #passer en tensor
        print("self.W_theta.shape : " )
        print(self.W_theta.shape)
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return torch.from_numpy(np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0))
    
    
#Tensor.numpy() pour commencer, à la fin repasser en tensor normaux.
        

In [50]:
class GNN_Model(nn.Module):
    def __init__(self):
        super(GNN_Model, self).__init__()
        # self.linear = nn.Sequential(
        #     DevConv(graph, 1),
        #     nn.ReLU(),
        #     DevConv(graph, 64),
        #     nn.ReLU(),
        #     DevConv(graph, 1),
        #     nn.Sigmoid()
        # )
        self.devconv = DevConv(graph,1)
        self.relu = nn.ReLU()
        self.devconv2 = DevConv(graph, 64)
        self.relu2 = nn.ReLU()
        self.devconv3 = DevConv(graph,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x= self.devconv(x)
        x= self.relu(x)
        x= self.devcon2(x)
        x= self.relu2(x)
        x= self.devconv3(x)
        x= self.sigmoid(x)
        return x

In [51]:
torch.manual_seed(0)  #  for repeatable results
gnn = GNN_Model()
# inp = np.array([[[[1,2,3,4],  # batch(=1) x channels(=1) x height x width
#                   [1,2,3,4],
#                   [1,2,3,4]]]])
inp = np.array([])
x = torch.tensor(inp, dtype=torch.float)
print('Forward computation thru model:', gnn(x))

self.W_theta.shape : 
torch.Size([1, 3])
self.W_theta.shape : 
torch.Size([1, 3])
self.W_theta.shape : 
torch.Size([1, 3])


TypeError: relu(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [29]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)
print("inclusion_score shape :")
print(inclusion_score.shape)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)
print("inclusion_score shape :")
print(inclusion_score.shape)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

self.W_theta.shape : 
torch.Size([1, 3])
[0.39470286 0.3949325  0.39470286 ... 0.36745507 0.53339097 0.36745507]
inclusion_score shape :
(5999,)
self.W_theta.shape : 
torch.Size([1, 3])
tensor([0.4738, 0.4740, 0.4738,  ..., 0.4411, 0.6402, 0.4411],
       dtype=torch.float64)
inclusion_score shape :
torch.Size([5999])
self.W_theta.shape : 
torch.Size([1, 3])


C:\Users\basti\AppData\Local\Temp\ipykernel_9064\237292033.py:33: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return torch.from_numpy(np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0))


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

### Multinomial Sampling

In [ ]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 0 0 ... 0 0 0]


In [ ]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(list(graph._node.keys()))[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[75.0252    , -0.66553295,  3.8       ],
       [75.01418   , -0.49935842, 16.7       ],
       [75.        ,  0.        ,  3.8       ],
       [75.00158   ,  0.16653232, 16.7       ],
       [75.0252    ,  0.66553295,  3.8       ]], dtype=float32)

# KNN extended graph

In [ ]:
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 45259


# Edge Predictor

In [ ]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(5999, 64)

In [ ]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [ ]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [ ]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [ ]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 30)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 8)	1
  (1, 9)	1
  (1, 10)	1
  :	:
  (5997, 5840)	1
  (5997, 5843)	1
  (5997, 5844)	1
  (5997, 5847)	1
  (5997, 5983)	1
  (5997, 5985)	1
  (5997, 5987)	1
  (5997, 5989)	1
  (5997, 5991)	1
  (5997, 5993)	1
  (5997, 5995)	1
  (5998, 5849)	1
  (5998, 5851)	1
  (5998, 5853)	1
  (5998, 5855)	1
  (5998, 5858)	1
  (5998, 5859)	1
  (5998, 5862)	1
  (5998, 5984)	1
  (5998, 5986)	1
  (5998, 5988)	1
  (5998, 5990)	1
  (5998, 5992)	1
  (5998, 5994)	1
  (5998, 5996)	1
[[0.06762367 0.06669079 0.06762367 ... 0.07336916 0.07336916 0.07336916]
 [1.06536342 0.07141035 0.07070604 ... 0.06638412 0.06638412 0.06638412]
 [1.01863433 0.48535413 0.06762367 ... 0.07336916 0.07336916 0.07336916]
 ...
 [1.15231462 1.08058119 1.15231462 ... 0.07142857 0.07142857 0.

In [ ]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [ ]:
print(A_s)  # symmétrique
print(A_s.shape)

[[0.00144363 0.00145854 0.00144815 ... 0.03488992 0.03464604 0.03486086]
 [0.00145854 0.00147732 0.00146473 ... 0.03481    0.03458935 0.03478371]
 [0.00144815 0.00146473 0.00145533 ... 0.03497431 0.03473044 0.03494525]
 ...
 [0.03488992 0.03481    0.03497431 ... 0.94490979 0.93845527 0.94447439]
 [0.03464604 0.03458935 0.03473044 ... 0.93845527 0.93649449 0.93844437]
 [0.03486086 0.03478371 0.03494525 ... 0.94447439 0.93844437 0.94415178]]
(5999, 5999)


# Face Classifier

### TriConv

#### Inputs

In [ ]:
nodes = list(extended_graph.nodes())
edges = list(extended_graph.edges())

igraph_g = igraphGraph(directed=extended_graph.is_directed())

igraph_g.add_vertices(nodes)
igraph_g.add_edges([(nodes.index(u), nodes.index(v)) for u, v in edges])
print(igraph_g.summary())

IGRAPH UN-- 5999 45259 -- 
+ attr: name (v)


In [ ]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[  37 4206 4207]
 [4191 4192 4193]
 [4176 4177 4178]]
126401


In [ ]:
triangles = np.array(igraph_g.vs['name'])[triangles_ids_igraph]
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[75.01686     0.66553295  3.7812707 ]
  [75.01482     0.66553295  3.7796104 ]
  [75.01586     0.66553295  3.7804136 ]]

 [[75.05917     1.3272538   3.7185588 ]
  [75.063354    1.3272538   3.7217672 ]
  [75.06736     1.3272538   3.72519   ]]

 [[75.13281     1.9813722   3.6171947 ]
  [75.142204    1.9813722   3.624396  ]
  [75.1512      1.9813722   3.6320791 ]]]
126401


In [ ]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(igraph_g.vs))
p_init[i] = (A_s_ij + A_s_ik + A_s_jk) / 3

#### Calculate barycenter

In [ ]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(126401, 3)
[[75.01585     0.66553295  3.7804317 ]
 [75.06329     1.3272538   3.7218387 ]
 [75.142075    1.9813722   3.6245565 ]]


#### KNN Tri

In [ ]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [ ]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(126401, 19, 3)


#### calculate e norm matrix

In [ ]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(126401, 3)
[[0.00263391 0.00131664 0.00131821]
 [0.00527009 0.01053509 0.00526864]]


#### Calculate r

In [ ]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(126401,)

In [ ]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(126401, 19)
(126401, 19)


In [ ]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(126401, 19, 5)


#### Calculate f

In [ ]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

1.0000000000000004
(5999,)
[0.00013181 0.00013181 0.00013181 ... 0.00013163 0.00013163 0.00013163]


C:\Users\ahdel\AppData\Local\Temp\ipykernel_10880\1047055924.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))


# Simplified Mesh

In [ ]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[:1])

(1999, 3, 3)
[[[79.978134   7.9794145  2.7418654]
  [80.633224   8.237194   2.9077842]
  [80.4243     8.237194   2.0375378]]]


In [ ]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 997
Number of edges: 2869


In [ ]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)